In [53]:
import os


os.environ["DB"] = "database/hotel_reservation_sample.db"

In [54]:
from data_access.hotel_dal import HotelDAL
from data_access.booking_dal import BookingDAL

hdal = HotelDAL()


# Testen, ob die Verbindung funktioniert
try:
    with hdal._connect() as conn:
        print("HotelDAL Verbindung erfolgreich hergestellt!")
except Exception as e:
    print(f"Fehler bei der Verbindung mit HotelDAL: {e}")

bdal = BookingDAL()
try:
    with bdal._connect() as conn:
        print("BookingDAL Verbindung erfolgreich hergestellt!")
except Exception as e:
    print(f"Fehler bei der Verbindung mit BookingDAL: {e}")

HotelDAL Verbindung erfolgreich hergestellt!
BookingDAL Verbindung erfolgreich hergestellt!


In [55]:

bookings = bdal.fetchall("SELECT * FROM booking")

if bookings:
    print("Alle Buchungen")
    for booking in bookings:
        print(booking)
else:
    print("Keine Buchungen gefunden")


Alle Buchungen
(1, 1, 1, datetime.date(2025, 6, 1), datetime.date(2025, 6, 5), 0, 1000.0)
(2, 2, 2, datetime.date(2025, 7, 10), datetime.date(2025, 7, 15), 0, 2000.0)
(3, 3, 3, datetime.date(2025, 8, 20), datetime.date(2025, 8, 22), 0, 1300.0)
(4, 4, 4, datetime.date(2025, 9, 5), datetime.date(2025, 9, 10), 1, 0.0)
(5, 5, 5, datetime.date(2025, 10, 1), datetime.date(2025, 10, 7), 0, 9000.0)


In [56]:
# 1.1. Ich möchte alle Hotels in einer Stadt durchsuchen, damit ich das Hotel nach meinem bevorzugten Standort (Stadt) auswählen kann.

hotels = bdal.fetchall("SELECT * FROM hotel LEFT JOIN address ON hotel.address_id = address.address_id WHERE address.city LIKE  '%Zürich%' ")

if hotels:
    print("Hotel gefunden")
    for hotel in hotels:
        print(hotel)
else:
    print("Hotel nicht gefunden")


Hotel gefunden
(1, 'Hotel Baur au Lac', 5, 1, 1, 'Bahnhofstrasse 1', 'Zürich', '8001')
